In [1]:
import os
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tldextract
import re

### Load Data

In [10]:
toxic_df = pd.read_csv("toxic.tsv",sep='\t')

In [2]:
tsv_df = pd.read_csv("../data/val.tsv",sep='\t')

In [2]:
train_file = '../data/train.jsonl'
validation_file = '../data/validation.jsonl'
train_df = pd.read_json(train_file, lines = True)
validation_df = pd.read_json(validation_file, lines = True)

train_df = train_df.append(validation_df)

C:\Users\Vi\AppData\Local\Temp\ipykernel_17544\1950742278.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append(validation_df)


In [3]:
## Transforming Data Step 1:
train_df = train_df.drop(['postId', 'targetMedia', 'provenance', 'spoiler', 'spoilerPositions', 'targetTitle'], axis=1)
#validation_df = validation_df.drop(['postId', 'targetMedia', 'provenance', 'targetDescription', 'targetTitle'], axis=1)

train_df['targetUrl'] = train_df['targetUrl'].astype(str)
#validation_df['targetUrl'] = validation_df['targetUrl'].astype(str)
train_df['targetUrl'] = train_df.targetUrl.apply(lambda x: '.'.join(tldextract.extract(x)[1:]))
#validation_df['targetUrl'] = validation_df.targetUrl.apply(lambda x: '.'.join(tldextract.extract(x)[1:]))

train_df = train_df.fillna("")
#validation_df = validation_df.fillna("")
train_df['targetDescription'] = train_df["targetDescription"].astype(str)
train_df['postText'] = train_df["postText"].astype(str)

In [4]:
## Calculated Measures
train_df["title_spoiler_ratio"] = train_df.apply(lambda x: len(x["targetDescription"].split(" ")) / len(x["postText"].split(" ")), axis=1 )
#train_df["first_spoiler"] = train_df.apply(lambda x: x["spoilerPositions"][0][0][0], axis=1)

string_list = []
for i, j in train_df.iterrows():
    string = ""
    for k in j["targetParagraphs"]:
        string += k + " "
    string_list.append(string)
train_df["full_context"] = string_list


In [5]:
## Enrich with NER data
ner_df  = pd.read_json("../data/ner/train_ner.json")
ner2_df = pd.read_json("../data/ner/validation_ner.json")

ner2_df = ner2_df.drop(['postText', 'postPlatform', "targetParagraphs", "targetTitle", "targetDescription", "targetKeywords", "targetMedia", "targetUrl", "provenance", "spoiler", "spoilerPositions", "tags", "full_context"], axis=1)
ner_df =  ner_df.append(ner2_df, ignore_index=True)

train_df = pd.merge(train_df, ner_df, on='uuid', how='left')

C:\Users\Vi\AppData\Local\Temp\ipykernel_17544\1617025903.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ner_df =  ner_df.append(ner2_df, ignore_index=True)


In [7]:
ner_df.tail()

,uuid,postId,ner_orgs,ner_persons,ner_dates,ner_locations
3995,1189d343-42eb-47e7-8395-ff978a683875,428006164904034305,[YouTube],"[Kyle, Josh]",[this week],[]
3996,7912282b-137b-4098-875d-8ad9f19354a8,806153730206892032,"[The New York Times, Politico, Harvard Univers...","[Suprun, Donald Trump, George W. Bush, Christo...","[each day, Sept. 11, 9/11, Dec. 19, two days]","[Texas, Ohio, America]"
3997,1fdf71e8-ec14-4c3b-a7c5-ca678c6f8ccb,847331053991813120,[Instagram],"[Instagram, Rachel Crawley ☾, Crawley, nomakeu...","[22-year-old, 2017, years, today, Mar 3, 2017,...",[Crawley]
3998,17f6b540-cf8d-4ddf-8321-1c9ce2315d71,788056531304583168,"[Reddit, CBS, Assange, Equador, Gizmodo, CNET,...","[Declan McCullagh, John Kerry, Assange, Roger ...","[June 2012, 16 October 2016, October 17, 2016,...","[UK, Ecuador, London, U.S.]"
3999,89dcad77-d8ad-4705-8676-717b26fda2ad,388308677494444032,"[Munch, HARGITAY, The Huffington Post, SVU, NBC]","[Belzer, Finn Wittrock, Matt DeCapua, John Mun...","[more than 20 years, Oct. 9, 1993, May 2013, 1...",[]


### Combine into Text

In [12]:
def combine_features(data_df):
    all_features = []
    labels = []

    for index, row in data_df.iterrows():
        """combined = ''
        combined += f'The post contains the title {row["postText"][1:-1]}. ' \
                    f'The spoiler has a length ratio of {row["title_spoiler_ratio"]}. '
                    #f'The post contains the spoiler {row["spoiler"][1:-1]}. ' \
                    #f'The spoiler first appears at paragraph {row["first_spoiler"]}. ' \

        if re.match(".*\d+\s*[\.\)].+\d+?\s*[\.\)].+?\d+\s*[\.\)]",row["full_context"], re.MULTILINE | re.IGNORECASE):
            combined += f'The spoiler contains an enumeration or multi-line. '
        
        if len(row["ner_orgs"]) > 0:
            combined += f'The context involves {len(row["ner_orgs"])} organisations. '
            #combined += f'The context contains the following organisations {str(row["ner_orgs"])[1:-1]}. '
        if len(row["ner_persons"]) > 0:
            combined += f'The context involves {len(row["ner_persons"])} persons. '
            #combined += f'The context contains the following persons {str(row["ner_persons"])[1:-1]}. '
        if len(row["ner_dates"]) > 0:
            combined += f'The context involves {len(row["ner_dates"])} dates. '
            #combined += f'The context contains the following dates {str(row["ner_dates"])[1:-1]}. '
        if len(row["ner_locations"]) > 0:
            combined += f'The context involves {len(row["ner_locations"])} locations. '
            #combined += f'The context contains the following locations {str(row["ner_locations"])[1:-1]}. '
            
        combined += f'The post was published on {row["postPlatform"]}. ' \
                    f'The post is sourced from the website {row["targetUrl"]}. '
        combined = combined.replace('"', "'")
        all_features.append(combined)
        labels.append(row["tags"][0])"""
        combined = ''
        combined += f'Title: {row["postText"][1:-1]}. ' \
                    f'Spoiler Length Ratio: {row["title_spoiler_ratio"]}. '
        if re.match(".*\d+\s*[\.\)].+\d+?\s*[\.\)].+?\d+\s*[\.\)]",row["full_context"], re.MULTILINE | re.IGNORECASE):
            combined += f'Enumeration or multi-line. '
        
        if len(row["ner_orgs"]) > 0:
            combined += f'{len(row["ner_orgs"])} organisations. '
            #combined += f'The context contains the following organisations {str(row["ner_orgs"])[1:-1]}. '
        if len(row["ner_persons"]) > 0:
            combined += f'{len(row["ner_persons"])} persons. '
            #combined += f'The context contains the following persons {str(row["ner_persons"])[1:-1]}. '
        if len(row["ner_dates"]) > 0:
            combined += f'{len(row["ner_dates"])} dates. '
            #combined += f'The context contains the following dates {str(row["ner_dates"])[1:-1]}. '
        if len(row["ner_locations"]) > 0:
            combined += f'{len(row["ner_locations"])} locations. '
            #combined += f'The context contains the following locations {str(row["ner_locations"])[1:-1]}. '
            
        combined += f'Publishing Platform: {row["postPlatform"]}. ' \
                    f'Source Website {row["targetUrl"]}. '
        combined = combined.replace('"', "'")
        all_features.append(combined)
        labels.append(row["tags"][0])


    return all_features, labels

all_features, labels = combine_features(train_df)

In [13]:
all_features[97]

"Title: 'One thing women would choose over sex that we're not even surprised about'. Spoiler Length Ratio: 2.1538461538461537. 2 persons. 4 dates. Publishing Platform: Twitter. Source Website huff.to. "

In [11]:
display(train_df.iloc[97])

uuid                                4cd4e1f1-7425-4f6e-b520-6335be81724c
postText               ["One thing women would choose over sex that w...
postPlatform                                                     Twitter
targetParagraphs       [Carving out time for yourself during the day ...
targetDescription      Carving out time for yourself during the day -...
targetKeywords         Love & Sex,things women prefer to sex,sex,the ...
targetUrl                                                        huff.to
tags                                                            [phrase]
title_spoiler_ratio                                             2.153846
full_context           Carving out time for yourself during the day -...
postId                                                399413489804275712
ner_orgs                                                              []
ner_persons                     [Celestial Seasonings, Christina Norman]
ner_dates                   [the day, each day, the

In [10]:
max(all_features, key=len)

"The post contains the title 'Daniel Craig Was Offered A Staggering Amount Of Money To Carry On Playing Bond'. The spoiler has a length ratio of 0.07142857142857142. The spoiler contains an enumeration or multi-line. The context involves 13 organisations. The context involves 14 persons. The context involves 9 dates. The context involves 5 locations. The post was published on Facebook. The post is sourced from the website James (Cash) Bond! Daniel Craig is 'offered a staggering $150million by Sony bosses for two more 007 movies'.. "

In [9]:
train_tsv = pd.DataFrame(list(zip(all_features, labels)),
              columns=['text','label'])

In [10]:
train_tsv.to_csv("../data/full_train_short.tsv", sep="\t", encoding='utf-8', index=False)

In [19]:
train_tsv[:3200].to_csv("../data/train.tsv", sep="\t", encoding='utf-8')

In [20]:
train_tsv[3200:].to_csv("../data/val.tsv", sep="\t", encoding='utf-8')